In [1]:
import obi_one as obi
import os
import pandas as pd

In [2]:
input_root = "/Users/pokorny/Data/Circuits"
output_root = "../../../obi-output/extracted_tiny_circuits"

In [3]:
N_ids = 10
node_selection_files = ["./top_rc_nodes_dim2_rc.csv",
                        "./top_nodes_dim6.csv"]
def sel_first_n(elements, n):
    assert len(elements) >= n, f"ERROR: Less than {n} elements to select from!"
    return elements[:n]
node_id_lists = [obi.NamedList(name=f"N_{N_ids}__" + os.path.splitext(os.path.split(_f)[1])[0], elements=tuple(sel_first_n(pd.read_csv(_f)["node_ids"].to_list(), N_ids))) for _f in node_selection_files]

In [4]:
circuit_extractions_form = obi.CircuitExtractions(
                    initialize=obi.CircuitExtractions.Initialize(
                        # circuit=obi.Circuit(name="ToyCircuit-S1-6k", path=os.path.join(input_root, "ToyCircuit-S1-6k", "circuit_config.json")),
                        circuit=obi.Circuit(name="nbS1-HEX0", path=os.path.join(input_root, "nbS1-HEX0", "circuit_config.json")),
                        run_validation=False,
                    ),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Excitatory", "Inhibitory"], random_sample=[100, 500]),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Layer2", "Layer3", "Layer4", "Layer5", "Layer6"], random_sample=10),
                    neuron_set=obi.IDNeuronSet(neuron_ids=node_id_lists)
)

grid_scan = obi.GridScan(form=circuit_extractions_form, output_root=output_root, coordinate_directory_option="VALUE")

In [5]:
# Display the intermediary data
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
grid_scan.coordinate_instances(display=True)


MULTIPLE VALUE PARAMETERS
neuron_set.neuron_ids: [N_10__top_rc_nodes_dim2_rc, N_10__top_nodes_dim6]

COORDINATE PARAMETERS
neuron_set.neuron_ids: N_10__top_rc_nodes_dim2_rc
neuron_set.neuron_ids: N_10__top_nodes_dim6

COORDINATE INSTANCES
CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'neuron_ids'], value=N_10__top_rc_nodes_dim2_rc)], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', neuron_set=IDNeuronSet(type='IDNeuronSet', simulation_level_name=None, random_sample=None, random_seed=0, neuron_ids=N_10__top_rc_nodes_dim2_rc), initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', name='nbS1-HEX0', path='/Users/pokorny/Data/Circuits/nbS1-HEX0/circuit_config.json'), run_validation=False))
CircuitE

[CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'neuron_ids'], value=N_10__top_rc_nodes_dim2_rc)], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', neuron_set=IDNeuronSet(type='IDNeuronSet', simulation_level_name=None, random_sample=None, random_seed=0, neuron_ids=N_10__top_rc_nodes_dim2_rc), initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', name='nbS1-HEX0', path='/Users/pokorny/Data/Circuits/nbS1-HEX0/circuit_config.json'), run_validation=False)),
 CircuitExtraction(idx=1, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanP

In [6]:
grid_scan.execute(processing_method='run')

Extracting subcircuit from 'nbS1-HEX0'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 888.42it/s]

Copied 10 morphologies into container (0 already existed)
Copying biophysical_neuron_models for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE
Extracting subcircuit from 'nbS1-HEX0'


Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1105.98it/s]

Copied 10 morphologies into container (0 already existed)
Copying biophysical_neuron_models for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE
create_bbp_workflow_campaign_config() not yet complete.


{0: None, 1: None}

In [7]:
# Check circuits
from bluepysnap import Circuit

for _inst in grid_scan.coordinate_instances():
    cfg = grid_scan.output_root / str(_inst.neuron_set.neuron_ids) / "circuit_config.json"
    c = Circuit(cfg)
    print(f"{_inst.neuron_set.neuron_ids}: {c.nodes['S1nonbarrel_neurons'].size} neurons, {c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size} synapses")


N_10__top_rc_nodes_dim2_rc: 10 neurons, 19 synapses
N_10__top_nodes_dim6: 10 neurons, 176 synapses
